# 본문 스크래핑

In [7]:
import pandas as pd
import numpy as np
import time
from tqdm.notebook import tqdm
import random
import urllib.request
from selenium.common.exceptions import NoSuchElementException, UnexpectedAlertPresentException, TimeoutException
from selenium import webdriver
from selenium.webdriver.common.by import By
# import chromedriver_auto


# 웹드라이버 설정
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)
# options.add_argument('headless') # 창 숨기기


ModuleNotFoundError: No module named 'selenium'

In [2]:
from user_agent import generate_user_agent, generate_navigator

print(generate_user_agent(device_type='desktop'))
print(generate_user_agent(os='win', device_type='desktop'))

navigator = generate_navigator()

random_user = generate_user_agent(os='win', device_type='desktop')
print(random_user)

Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.111 Safari/537.36
Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 10.0; Win64; x64; Trident/6.0)
Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.2; Trident/5.0)


In [3]:
from selenium.webdriver.chrome.options import Options

chrome_options = Options()
chrome_options.add_argument(random_user)
driver = webdriver.Chrome(executable_path ='./chromedriver_win32/chromedriver.exe',options = chrome_options)

driver.maximize_window()
driver.get("https://www.naver.com/")
driver.implicitly_wait(10)


C:\Users\user\AppData\Local\Temp\ipykernel_12492\1335855278.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path ='./chromedriver_win32/chromedriver.exe',options = chrome_options)


In [4]:
url_df = pd.read_csv("./나무문화/2020/나무문화_링크_2020_9월.csv", encoding='UTF-8')
print(url_df.shape, url_df.columns)
url_df


(11904, 3) Index(['url', 'title', 'date'], dtype='object')


,url,title,date
0,https://blog.naver.com/jinidag/222104033228,추석연휴 가볼만한곳 한국민속촌 야외 당일치기 나들이,2020.09.30.
1,https://blog.naver.com/taya2765/222104032204,"동래 가볼만한 곳, 복천박물관",2020.09.30.
2,https://blog.naver.com/yeonjae1213/222104028784,배꼽 빠질 뻔했다,2020.09.30.
3,https://blog.naver.com/sunnybridge_/222104026647,"내가 사랑했던 마을,탬워스(Tamworth)",2020.09.30.
4,https://blog.naver.com/lby56/222104022581,언택트 한가위… ‘꼰대’ 삼촌이 10대 조카에게 보내는 편지,2020.09.30.
...,...,...,...
11899,https://blog.naver.com/chookk7/222076688184,“먹튀자본 MBK 김병주의 땅투기로 안산이 망가진다”,2020.09.01.
11900,https://blog.naver.com/bugman1303/222076686202,머리가 좋은 동물들,2020.09.01.
11901,https://blog.naver.com/sherry8279/222076052955,"<도시는 무엇으로 사는 가, 유현준> 도시의 다채로운 생각과...",2020.09.01.
11902,https://blog.naver.com/butterfly0724/222071335341,[경기도 미술관 카페] 호이안,2020.09.01.


In [5]:
blog_links = url_df['url'].to_list()
print(len(blog_links))


11904


In [6]:
# df = pd.DataFrame(columns = {'url','본문'}) # 처음 만들때를 위한 코드라 주석처리 하였음.
# df.to_csv("./나무문화/2020/나무문화_본문_2020_9월.csv", index=False, encoding='UTF-8')
# df.to_excel("./나무문화/2020/나무문화_본문_2020_9월.xlsx", index=False, encoding='UTF-8')

In [10]:
df = pd.read_csv("./나무문화/2020/나무문화_본문_2020_9월.csv", encoding='UTF-8')
df

,본문,url
0,"추석연휴 가볼만한곳 야외 나들이\n""이글은 한국민속촌으로부터 금전적 지원을 받아 작...",https://blog.naver.com/jinidag/222104033228
1,복천박물관\n부산 동래구 복천로 63\n매일 09:00-18:00 (매주 마지막주 ...,https://blog.naver.com/taya2765/222104032204
2,오늘 아침에 일어나서 엄마랑 극적 화해하고 기분 좋았다. 그런데 기상은 늦었다 10...,https://blog.naver.com/yeonjae1213/222104028784
3,호주 워킹홀리데이 상반기 시즌은 사진이 많이 없어서\n포스팅 할게 많이 없다 ㅠㅠ\...,https://blog.naver.com/sunnybridge_/222104026647
4,블로그\n공지 목록\n공지글\n글 제목 작성일\n(12)\n공지 이병렬 저서 출간 ...,https://blog.naver.com/lby56/222104022581
...,...,...
2246,블로그\n성경인물 40개의 글\n성경인물\n목록열기\n아볼로 / 위대한 설교가 / ...,https://blog.naver.com/noemisuh/222099203708
2247,"한서남궁억기념관, 독립을 향한 구국일념이 서린 홍천가볼만한곳\n대한민국 대표 #건강...",https://blog.naver.com/hcgnews/222099202635
2248,청정 거창 가을하늘이 너무 아름다운 요즘입니다.\n아침저녁 쌀쌀한 기온차이로 푸른하...,https://blog.naver.com/geochanggun/222099202787
2249,진천 가볼 만한 곳\n비 갠 오후 문학의 정취 속 즐기는 산책 야외 문학공원\n비 ...,https://blog.naver.com/jincheongun/222099198163


In [ ]:
len(df)

In [12]:
#### 삭제/비공개 링크 인덱스도 매번 바꿔줄 것!!! ###
# contents = []
# old_blogs = []
hide_link = []

for i in tqdm(blog_links[len(df):]):
    content_dic = {'url' : [], '본문' : []}
    content_dic['url'].append(i)
    driver.get(i)
    driver.implicitly_wait(10)
    try:
        driver.switch_to.frame("mainFrame")
    except TimeoutException as e:
        print('시간 초과 오류 : ', e)
        content_dic['본문'].append('수집불가능')
    except UnexpectedAlertPresentException as e:
    ### 이 오류 해결함. 에러 코드 파일에서 직접 임포트하면 된다!
        print(e)
                                                                            # 여기도 바꿔야 함
        print('{}번째 게시글은 비공개로 전환되었거나 게시판이 바뀜.'.format(blog_links.index(i)))
        hide_link.append(i)
        time.sleep(1.5)
        driver.switch_to.frame('mainFrame')
    time.sleep(1)
    try:
        a = driver.find_element(By.CSS_SELECTOR,'div.se-main-container').text
        content_dic['본문'].append(a)
    # NoSuchElement 오류시 예외처리(구버전 블로그에 적용)
    except NoSuchElementException:
        a = driver.find_element(By.CSS_SELECTOR,'div#content-area').text
        content_dic['본문'].append(a)
        #old_blogs.append(i)
    additional_df = pd.DataFrame(content_dic)
    df = df.append(additional_df, ignore_index = True)
    df.to_csv("./나무문화/2020/나무문화_본문_2020_9월.csv", index=False, encoding='UTF-8')
    df.to_excel("./나무문화/2020/나무문화_본문_2020_9월.xlsx", index=False, encoding='UTF-8')
    print(blog_links.index(i), df['본문'].tolist().index(a))
    # if blog_links.index(i) != df['본문'].tolist().index(a):
    #     break


  0%|          | 0/9653 [00:00<?, ?it/s]

C:\Users\user\AppData\Local\Temp\ipykernel_12492\2098576090.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(additional_df, ignore_index = True)
c:\Users\user\anaconda3\lib\site-packages\xlsxwriter\worksheet.py:1170: UserWarning: Ignoring URL 'https://youtu.be/d7MAV_Jz06g
황성옛터%20-%20남인수
https://youtu.be/YhZuirepbeo
황성옛터’%20가수%20이애리수%20씨%20별세%20향년%2099세
황성옛터에%20밤이%20되니%20월색만%20고요해
폐허에%20서린%20회포를%20말하여%20주노나
아%20~%20외로운%20저%20나그네%20홀로이%20잠못이뤄
구슬픈%20버래소래에%20말없이%20눈물져요
흘러간%20민족가요%20“황성옛터”를%20모르는%20사람은%20없을%20것입니다.
그러나%20이%20노래를%20처음%20부른%20가수를%20아는%20사람은%20드물%20것입니다.
2008년%2012월%205일%20밤%20자정%20무렵에%20우연히%20FM%20라디오에서%20국내%20신문사%20최초로%20연예부를%20창설하고%20연예기자%201호로%20자리매김하고%20있는%20한국일보의%20정홍택%20기자가%20“황성옛터”%20가요에%20얽힌%20취재담을%20듣고%20필자도%20처음%20듣는%20귀한%20이야기라%20주섬주섬%20정리한%20기록이%20있어%20소개%20합니다.
“황성옛터”는%20한국인%20왕평이%20작사하고%20전수린%20작곡으로%20이애리수(李愛利秀)%20가수가%20불렀다고%20합니다.%20그리고%20한국인의%20심금을%20울리는

2251 2251
2252 2252
2253 2253
2254 2254
2255 2255
2256 2256
2257 2257
2258 2258
2259 2259
2260 2260
2261 2261
2262 2262
2263 2263
2264 2264
2265 2265
2266 2266
2267 2267
2268 2268
2269 2269
2270 2270
2271 2271
2272 2272
2273 2273
2274 2274
2275 2275
2276 2276
2277 2277
2278 2278
2279 2279
2280 2280
2281 2281
2282 2282
2283 2283
2284 2284
2285 2285
2286 2286
2287 2287
2288 2288
2289 2289
2290 2290
2291 2291
2292 2292
2293 2293
2294 2294
2295 2295
2296 2296
2297 2297
2298 2298
2299 2299
2300 2300
2301 2301
2302 2302
2303 2303
2304 2304
2305 2305
2306 2306
2307 2307
2308 2308
2309 2309
2310 2310
2311 2311
2312 2312
2313 2313
2314 2314
2315 2315
2316 2316
2317 2317
2318 2318
2319 2319
2320 2320
2321 2321
2322 2322
2323 2323
2324 2324
2325 2325
2326 2326
2327 2327
2328 2328
2329 2329
2330 2330
2331 2331
2332 2332
2333 2333
2334 2334
2335 2335
2336 2336
2337 2337
2338 2338
2339 2339
2340 2340
2341 2341
2342 2342
2343 2343
2344 2344
2345 2345
2346 2346
2347 2347
2348 2348
2349 2349
2350 2350


c:\Users\user\anaconda3\lib\site-packages\xlsxwriter\worksheet.py:1170: UserWarning: Ignoring URL 'http://thetravelnews.co.kr/08/483272/
제2의%20올레길을%20꿈꾸는%20남해%20바래길
%5b여행테라피스트%20김홍덕기자%5d%20코로나19%20여파로%20인해%20도보여행의%20수요가%20늘고%20있는%20가운데%20지자체들이%20저마다%20걷기%20좋은%20길%20알리기에%20나섰다%20.경상남도%20남해군%20역시%20예외가%20아니어서%2010년%20전에%20개통했던%20바래길을%20새로이%20단장하는%20작업에%20한창이다.%20이에%20따라%20바래길%202.0%20사업은%20전면적인%20리모델링을%20통해%20체류형%20방문지,%20해양%20관광지로서의%20위상으로%20탈바꿈하기%20위해%20새로이%20디자인%20가이드라인을%20설정하고%20안내서비스를%20고도화하며%20완보인증%20프로그램%20운영%20등을%20%5b…%5d
thetravelnews.co.kr
%5b여행테라피스트%20김홍덕기자%5d%20코로나19%20여파로%20인해%20도보여행의%20수요가%20늘고%20있는%20가운데%20지자체들이%20저마다%20걷기%20좋은%20길%20알리기에%20나섰다%20.경상남도%20남해군%20역시%20예외가%20아니어서%2010년%20전에%20개통했던%20바래길을%20새로이%20단장하는%20작업에%20한창이다.
이에%20따라%20바래길%202.0%20사업은%20전면적인%20리모델링을%20통해%20체류형%20방문지,%20해양%20관광지로서의%20위상으로%20탈바꿈하기%20위해%20새로이%20디자인%20가이드라인을%20설정하고%20안내서비스를%20고도화하며%20완보인증%20프로그램%20운영%20등을%20추진한다.
특히%20체류형%20관광%20플랫폼으로의%20전환을%20위해%203개%20코스의%20지선를%20개발하며%20기존%20노선도%20조정한다.%20

2439 2439
2440 2440
2441 2441
2442 2442
2443 2443


KeyboardInterrupt: 

In [17]:
df

,url,본문
0,https://blog.naver.com/jinidag/222104033228,"추석연휴 가볼만한곳 야외 나들이\n""이글은 한국민속촌으로부터 금전적 지원을 받아 작..."
1,https://blog.naver.com/taya2765/222104032204,복천박물관\n부산 동래구 복천로 63\n매일 09:00-18:00 (매주 마지막주 ...
2,https://blog.naver.com/yeonjae1213/222104028784,오늘 아침에 일어나서 엄마랑 극적 화해하고 기분 좋았다. 그런데 기상은 늦었다 10...
3,https://blog.naver.com/sunnybridge_/222104026647,호주 워킹홀리데이 상반기 시즌은 사진이 많이 없어서\n포스팅 할게 많이 없다 ㅠㅠ\...
4,https://blog.naver.com/lby56/222104022581,블로그\n공지 목록\n공지글\n글 제목 작성일\n(12)\n공지 이병렬 저서 출간 ...
5,https://blog.naver.com/qkrtiger/222104017905,컨디션 회복을 위한(?)\n걷기 ...\n아점 후\n형산강변 어싱 ㄱㄱ\n유강방면으...
6,https://blog.naver.com/tnqls11004/222104016240,안녕하세요. 강서구 SNS 서포터즈 '새로미 프렌즈'의 뚜비입니다! 오늘은 강서구에...
7,https://blog.naver.com/019088/222104016283,서로이웃 추가\n이번 달에 새롭게 추가되신 분들은 총 15분입니다. 블로그 운영하는...
8,https://blog.naver.com/jjggrr202/222104015943,blog\n일본 성지순례기(2018) 16개의 글\n일본 성지순례기(2018)\n목...


In [9]:
print(len(contents))
#print(len(old_blogs))
print(len(hide_link))


214
28
0
